In [ ]:
myrdd1=sc.textFile("/bigdatads/common_folder/assignment3/car_data_train.csv").map(lambda line: line.split(","))
header=myrdd1.first()
myrdd1=myrdd1.filter(lambda line:line!=header)
col=['make','model','priceUSD','year','condition','mileage(kilometers)','fuel_type','volume(cm3)','color','transmission','drive_unit']
train_data=myrdd1.toDF(col)
myrdd2=sc.textFile("/bigdatads/common_folder/assignment3/car_data_test.csv").map(lambda line: line.split(","))
header=myrdd2.first()
myrdd2=myrdd2.filter(lambda line:line!=header)
test_data=myrdd2.toDF(col)


In [ ]:
from pyspark.sql.types import DoubleType
train_data=train_data.withColumn('priceUSD',train_data['priceUSD'].cast(DoubleType())).withColumn('mileage(kilometers)',train_data['mileage(kilometers)'].cast(DoubleType())).withColumn('volume(cm3)',train_data['volume(cm3)'].cast(DoubleType()))
test_data=test_data.withColumn('priceUSD',test_data['priceUSD'].cast(DoubleType())).withColumn('mileage(kilometers)',test_data['mileage(kilometers)'].cast(DoubleType())).withColumn('volume(cm3)',test_data['volume(cm3)'].cast(DoubleType()))
train_data=train_data.drop('year','model','color')
test_data=test_data.drop('year','model','color')


In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
SI_make=StringIndexer(inputCol='make',outputCol='make_Index')
SI_condition=StringIndexer(inputCol='condition',outputCol='condition_Index')
SI_fuel_type=StringIndexer(inputCol='fuel_type',outputCol='fuel_type_Index')
SI_transmission=StringIndexer(inputCol='transmission',outputCol='transmission_Index')
SI_drive_unit=StringIndexer(inputCol='drive_unit',outputCol='drive_unit_Index')
train_data_SI=SI_make.fit(train_data).transform(train_data)
test_data_SI=SI_make.fit(test_data).transform(test_data)
train_data_SI=SI_condition.fit(train_data_SI).transform(train_data_SI)
test_data_SI=SI_condition.fit(test_data_SI).transform(test_data_SI)
train_data_SI=SI_fuel_type.fit(train_data_SI).transform(train_data_SI)
test_data_SI=SI_fuel_type.fit(test_data_SI).transform(test_data_SI)
train_data_SI=SI_transmission.fit(train_data_SI).transform(train_data_SI)
test_data_SI=SI_transmission.fit(test_data_SI).transform(test_data_SI)
train_data_SI=SI_drive_unit.fit(train_data_SI).transform(train_data_SI)
test_data_SI=SI_drive_unit.fit(test_data_SI).transform(test_data_SI)
train_data_SI.select('make','make_Index','condition','condition_Index','fuel_type','fuel_type_Index','transmission','transmission_Index','drive_unit','drive_unit_Index').show(10)
test_data_SI.select('make','make_Index','condition','condition_Index','fuel_type','fuel_type_Index','transmission','transmission_Index','drive_unit','drive_unit_Index').show(10)


In [ ]:
OHE_make=OneHotEncoder(inputCol='make_Index',outputCol='make_OHE')
OHE_condition=OneHotEncoder(inputCol='condition_Index',outputCol='condition_OHE')
OHE_fuel_type=OneHotEncoder(inputCol='fuel_type_Index',outputCol='fuel_type_OHE')
OHE_transmission=OneHotEncoder(inputCol='transmission_Index',outputCol='transmission_OHE')
OHE_drive_unit=OneHotEncoder(inputCol='drive_unit_Index',outputCol='drive_unit_OHE')
train_data_SI=OHE_make.transform(train_data_SI)
test_data_SI=OHE_make.transform(test_data_SI)
train_data_SI=OHE_condition.transform(train_data_SI)
test_data_SI=OHE_condition.transform(test_data_SI)
train_data_SI=OHE_fuel_type.transform(train_data_SI)
test_data_SI=OHE_fuel_type.transform(test_data_SI)
train_data_SI=OHE_transmission.transform(train_data_SI)
test_data_SI=OHE_transmission.transform(test_data_SI)
train_data_SI=OHE_drive_unit.transform(train_data_SI)
test_data_SI=OHE_drive_unit.transform(test_data_SI)
train_data_SI.select('make','make_Index','make_OHE','condition','condition_Index','condition_OHE','fuel_type','fuel_type_Index','fuel_type_OHE','transmission','transmission_Index','transmission_OHE','drive_unit','drive_unit_Index','drive_unit_OHE').show(10)
test_data_SI.select('make','make_Index','make_OHE','condition','condition_Index','condition_OHE','fuel_type','fuel_type_Index','fuel_type_OHE','transmission','transmission_Index','transmission_OHE','drive_unit','drive_unit_Index','drive_unit_OHE').show(10)


In [ ]:
from pyspark.ml.feature import VectorAssembler
assembler=VectorAssembler(inputCols=['make_Index','make_OHE','priceUSD','condition_Index','condition_OHE','mileage(kilometers)','fuel_type_Index','fuel_type_OHE','volume(cm3)','transmission_Index','transmission_OHE','drive_unit_Index','drive_unit_OHE'],outputCol='vector')
train_data_SI=train_data_SI.na.fill(value=0)
test_data_SI=test_data_SI.na.fill(value=0)
final_train_data=assembler.transform(train_data_SI)
final_test_data=assembler.transform(test_data_SI)
final_train_data.select('vector').show()
final_test_data.select('vector').show()


In [ ]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml import Pipeline
train_data=train_data.withColumnRenamed('priceUSD','label')
test_data=test_data.withColumnRenamed('priceUSD','label')
stage_1=StringIndexer(inputCol= 'make', outputCol= 'make_Index')
stage_2=StringIndexer(inputCol= 'condition', outputCol= 'condition_Index')
stage_3=StringIndexer(inputCol= 'fuel_type', outputCol= 'fuel_type_Index')
stage_4=StringIndexer(inputCol= 'transmission', outputCol= 'transmission_Index')
stage_5=StringIndexer(inputCol= 'drive_unit', outputCol= 'drive_unit_Index')
stage_6=OneHotEncoder(inputCol=stage_1.getOutputCol(),outputCol='make_encoded')
stage_7=OneHotEncoder(inputCol=stage_2.getOutputCol(),outputCol='condition_encoded')
stage_8=OneHotEncoder(inputCol=stage_3.getOutputCol(),outputCol='fuel_type_encoded')
stage_9=OneHotEncoder(inputCol=stage_4.getOutputCol(),outputCol='transmission_encoded')
stage_10=OneHotEncoder(inputCol=stage_5.getOutputCol(),outputCol='drive_unit_encoded')
stage_11=VectorAssembler(inputCols=['make_encoded','condition_encoded','mileage(kilometers)','fuel_type_encoded','volume(cm3)','transmission_encoded','drive_unit_encoded'],outputCol='features')
lr=LinearRegression(featuresCol='features',labelCol='label')
pipeline=Pipeline(stages=[stage_1,stage_2,stage_3,stage_4,stage_5,stage_6,stage_7,stage_8,stage_9,stage_10,stage_11,lr])
train_data_pipe=pipeline.fit(train_data)
test_data_pipe=pipeline.fit(test_data)
paramGrid=ParamGridBuilder().addGrid(lr.maxIter,[10,15]).addGrid(lr.regParam,[0.5,0.5]).build()
crossval=CrossValidator(estimator=pipeline,estimatorParamMaps=paramGrid,evaluator=modelEvaluator)
crossval=crossval.fit(train_data)
crossval_test=crossval.fit(test_data)
y_predict=crossval_test.transform(test_data)
modelEvaluator=RegressionEvaluator(predictionCol="prediction",labelCol="label", metricName="rmse")
print('RMSE:', modelEvaluator.evaluate(y_predict))
